# Kieran Molloy UCDPA Project 
### Course CIDAB 2022-01-18
***

### Environment Information
<table align="left">
<tr>
    <th>Environment Type</th>
    <th>Anaconda Version</th>
    <th>Anaconda Build Channel</th>
    <th>Python Version</th>
</tr>
<tr>
    <td>Anaconda </td>
    <td>2021.11 </td>
    <td>py39_0 </td>
    <td>3.9.7 </td>
</tr>
</table>

## Links to Kaggle Datasource: 
***

#### _[Transactional Retail Dataset of Electronics Store](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store)_

*  _[dirty_data.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=dirty_data.csv)_
*  _[missing_data.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=missing_data.csv)_
*  _[warehouses.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=warehouses.csv)_


## Install Modules

In [910]:
# %pip install geopy

## Import Modules
***

In [911]:
from matplotlib import pyplot as plt
from IPython.display import display 

import pandas as pd
import numpy as np
import seaborn as sns

from datetime import datetime

from geopy.distance import geodesic

## Set Variables
***

In [912]:
dirty_data = 'dirty_data.csv'
missing_data = 'missing_data.csv'
warehouses = 'warehouses.csv'

## Functions
***

In [913]:
def get_missing_nearest_warehouse(lat , Long ):
    ''' use coordinates to find the closest warehouse (clrow flies)'''
    
    # customer coordinates are provided
    coords_customer = lat, Long
    
    # get warehouse latitude and longitide from warehouse table
    coords_Nickolson = df_warehouses['lat'].loc[df_warehouses.index[0]], df_warehouses['lon'].loc[df_warehouses.index[0]]
    coords_Thompson = df_warehouses['lat'].loc[df_warehouses.index[1]], df_warehouses['lon'].loc[df_warehouses.index[1]]
    coords_Bakers = df_warehouses['lat'].loc[df_warehouses.index[2]], df_warehouses['lon'].loc[df_warehouses.index[2]]
    
    # use geodesic to do a km distance comparison
    Nickolson_cust_dist = geodesic(coords_customer, coords_Nickolson).km
    Thompson_cust_dist = geodesic(coords_customer, coords_Thompson).km
    Bakers_cust_dist = geodesic(coords_customer, coords_Bakers).km
    
    val = (Nickolson_cust_dist ,Thompson_cust_dist ,Bakers_cust_dist)
    
    x = val.index(min(val))
    
    if x == 0:
        warehouse = 'Nickolson'
        distance = Nickolson_cust_dist
        return(warehouse , distance)
    elif x == 1:
        warehouse = 'Thompson'
        distance = Nickolson_cust_dist
        return(warehouse , distance)
    else: 
        warehouse = 'Bakers'
        distance = Nickolson_cust_dist
        return(str(warehouse) , distance)

In [914]:
def get_missing_order_price(order_total, delivery_charges, coupon_discount):
    ''' calculate order_price from order_total, delivery_charges and coupon_discount '''
    order_price = (order_total - delivery_charges) / (100-coupon_discount) * 100
    return(round(order_price, 2))

In [915]:
def get_missing_order_total(order_price, delivery_charges, coupon_discount):
    ''' calculate order_total from order_price, delivery_charges and coupon_discount'''
    order_total = order_price - (order_price / 100 * coupon_discount) + delivery_charges
    return(round(order_total, 2))

In [916]:
def get_missing_season(date):
    '''calculate season from date'''
    # Spring = months 9,10,11
    # summer = months 12,1,2
    # Autumn = months 3,4,5
    # Winter = months 6,7,8

    month = date.month

    if month in [9,10,11]:
        season = 'Spring'
    elif month in [12,1,2]:
        season = 'Summer'
    elif month in [3,4,5]:
        season = 'Autumn'
    else:
        season = 'Winter'
    
    return(season)

## CSV import 
***

In [917]:
# read the CSV from the local Jupyter Notebook directory 
df_dirty_data = pd.read_csv(dirty_data)
df_missing_data = pd.read_csv(missing_data)
df_warehouses = pd.read_csv(warehouses)

## Initial CSV Review
***

### df_dirty_data dataframe review

In [918]:
type(df_dirty_data)

pandas.core.frame.DataFrame

In [919]:
print(df_dirty_data.columns)

Index(['order_id', 'customer_id', 'date', 'nearest_warehouse', 'shopping_cart',
       'order_price', 'delivery_charges', 'customer_lat', 'customer_long',
       'coupon_discount', 'order_total', 'season', 'is_expedited_delivery',
       'distance_to_nearest_warehouse', 'latest_customer_review',
       'is_happy_customer'],
      dtype='object')


In [920]:
df_dirty_data.head(10)

order_id   customer_id        date nearest_warehouse  \
0  ORD182494  ID6197211592  2019-06-22          Thompson   
1  ORD395518  ID0282825849  2019-12-29          Thompson   
2  ORD494479  ID0579391891  2019-03-02         Nickolson   
3  ORD019224  ID4544561904  2019-01-12         Nickolson   
4  ORD104032  ID6231506320  2019-11-28         Nickolson   
5  ORD146760  ID0311654900  2019-09-16            Bakers   
6  ORD337984  ID3394768956  2019-09-14          Thompson   
7  ORD072312  ID0774517121  2019-05-23          Thompson   
8  ORD377837  ID4769265355  2019-10-09            Bakers   
9  ORD462194  ID5301568579  2019-03-21          Thompson   

                                       shopping_cart  order_price  \
0  [('Lucent 330S', 1), ('Thunder line', 2), ('iS...        12200   
1        [('Thunder line', 1), ('Universe Note', 2)]         9080   
2               [('Thunder line', 1), ('pearTV', 2)]        10670   
3  [('Universe Note', 1), ('Alcon 10', 2), ('Oliv...        24800   
4  [('Universe Note', 1), ('Olivia x460', 1), ('i...         9145   
5        [('Thunder line', 2), ('Universe Note', 1)]         7810   
6  [('Candle Inferno', 1), ('Alcon 10', 1), ('Tos...        13700   
7  [('Universe Note', 1), ('Thunder line', 2), ('...         7960   
8  [('Alcon 10', 2), ('Thunder line', 1), ('Candl...        25390   
9  [('Universe Note', 1), ('Lucent 330S', 1), ('T...        13320   

   delivery_charges  customer_lat  customer_long  coupon_discount  \
0             79.89    -37.815105     144.932843               10   
1             62.71    -37.802736     144.951118                0   
2             65.87    -37.821302     144.957581               10   
3             57.61    -37.811416     144.973073               15   
4             75.54     37.823859     144.969892               25   
5             71.22     37.820245     145.014944               10   
6             74.84    -37.807743     144.951570                5   
7             52.28    -37.806337     144.959544                5   
8            107.58    -37.810810     145.014073               10   
9             62.26    -37.808675     144.942337               15   

   order_total  season  is_expedited_delivery  distance_to_nearest_warehouse  \
0     11059.89  Winter                   True                         1.2800   
1      9142.71  Summer                  False                         1.1621   
2      9668.87  Autumn                  False                         1.0949   
3     21137.61  Summer                  False                         0.8571   
4      6934.29  Spring                  False                         0.5867   
5      7100.22  Spring                  False                         2.0752   
6     13089.84  Spring                  False                         0.6767   
7     10789.79  Autumn                  False                         1.3043   
8     22958.58  Spring                   True                         1.6595   
9     11384.26  winter                   True                         0.6093   

                              latest_customer_review  is_happy_customer  
0  perfect phone and trusted seller. phone itself...               True  
1  it keeps dropping calls the wifi don't work th...              False  
2            five stars this is a great cheap phone.               True  
3        charger did not fit the charger didn't fit.              False  
4                                    four stars good               True  
5  stolen phone sold us a stolen phone so we coul...              False  
6  love our inferno stick,easy to set up and have...               True  
7                it sucks mine came with dead pixels              False  
8  this is how top phone should look like! super ...               True  
9  does not live up to its reputation. customer s...              False

In [921]:
# using display for better formatting in Notebook , use print usually
display(df_dirty_data.describe())

order_price  delivery_charges  customer_lat  customer_long  \
count     500.000000        500.000000    500.000000     500.000000   
mean    25522.216000         76.658200    -35.835234     144.969494   
std     86333.729169         14.481465     12.045393       0.022720   
min       585.000000         46.350000    -37.827123     144.924967   
25%      7050.000000         65.982500    -37.818222     144.953488   
50%     12807.500000         76.310000    -37.812165     144.965357   
75%     20360.000000         82.555000    -37.805364     144.983985   
max    947691.000000        114.040000     37.826339     145.019837   

       coupon_discount   order_total  distance_to_nearest_warehouse  
count       500.000000  5.000000e+02                     500.000000  
mean         10.890000  3.920967e+04                       2.204224  
std           8.649134  2.741940e+05                       8.812416  
min           0.000000  6.392900e+02                       0.107800  
25%           5.000000  6.454735e+03                       0.751425  
50%          10.000000  1.129396e+04                       1.030100  
75%          15.000000  1.811919e+04                       1.408625  
max          25.000000  5.688270e+06                      94.973400

In [922]:
print(df_dirty_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       500 non-null    object 
 1   customer_id                    500 non-null    object 
 2   date                           500 non-null    object 
 3   nearest_warehouse              500 non-null    object 
 4   shopping_cart                  500 non-null    object 
 5   order_price                    500 non-null    int64  
 6   delivery_charges               500 non-null    float64
 7   customer_lat                   500 non-null    float64
 8   customer_long                  500 non-null    float64
 9   coupon_discount                500 non-null    int64  
 10  order_total                    500 non-null    float64
 11  season                         500 non-null    object 
 12  is_expedited_delivery          500 non-null    boo

In [923]:
print(df_dirty_data.shape)

(500, 16)


In [924]:
print(df_dirty_data.isna().any())

order_id                         False
customer_id                      False
date                             False
nearest_warehouse                False
shopping_cart                    False
order_price                      False
delivery_charges                 False
customer_lat                     False
customer_long                    False
coupon_discount                  False
order_total                      False
season                           False
is_expedited_delivery            False
distance_to_nearest_warehouse    False
latest_customer_review           False
is_happy_customer                False
dtype: bool


In [925]:
print(df_dirty_data.isnull().sum())

order_id                         0
customer_id                      0
date                             0
nearest_warehouse                0
shopping_cart                    0
order_price                      0
delivery_charges                 0
customer_lat                     0
customer_long                    0
coupon_discount                  0
order_total                      0
season                           0
is_expedited_delivery            0
distance_to_nearest_warehouse    0
latest_customer_review           0
is_happy_customer                0
dtype: int64


### df_missing_data dataframe review

In [926]:
type(df_missing_data)

pandas.core.frame.DataFrame

In [927]:
print(df_missing_data.columns)

Index(['order_id', 'customer_id', 'date', 'nearest_warehouse', 'shopping_cart',
       'order_price', 'delivery_charges', 'customer_lat', 'customer_long',
       'coupon_discount', 'order_total', 'season', 'is_expedited_delivery',
       'distance_to_nearest_warehouse', 'latest_customer_review',
       'is_happy_customer'],
      dtype='object')


In [928]:
df_missing_data.head(10)

order_id   customer_id        date nearest_warehouse  \
0  ORD382112  ID0289597187  2019-03-06          Thompson   
1  ORD378488  ID1668523020  2019-05-05          Thompson   
2  ORD279446  ID0370751503  2019-03-24            Bakers   
3  ORD277196  ID0634774947  2019-01-12          Thompson   
4  ORD116193  ID3313210924  2019-09-02         Nickolson   
5  ORD005004  ID0472236192  2019-05-07         Nickolson   
6  ORD296379  ID0591306178  2019-01-19         Nickolson   
7  ORD447851  ID0591430562  2019-02-07               NaN   
8  ORD078449  ID0030287324  2019-10-20         Nickolson   
9  ORD018258  ID0245537598  2019-05-27         Nickolson   

                                       shopping_cart  order_price  \
0             [('pearTV', 1), ('Candle Inferno', 2)]       7170.0   
1  [('Thunder line', 1), ('Lucent 330S', 2), ('Al...      13590.0   
2      [('Candle Inferno', 2), ('Universe Note', 1)]       4310.0   
3  [('pearTV', 1), ('Thunder line', 1), ('Olivia ...      10940.0   
4  [('pearTV', 2), ('Universe Note', 2), ('Thunde...      21700.0   
5          [('Thunder line', 2), ('Lucent 330S', 2)]       6820.0   
6               [('Thunder line', 1), ('pearTV', 1)]       8490.0   
7  [('Alcon 10', 2), ('Toshika 750', 2), ('Univer...      32440.0   
8          [('Olivia x460', 2), ('iAssist Line', 2)]       6900.0   
9  [('Universe Note', 1), ('Lucent 330S', 1), ('i...       7205.0   

   delivery_charges  customer_lat  customer_long  coupon_discount  \
0             61.72    -37.811782     144.951972               15   
1             65.20    -37.808236     144.942181               10   
2             78.75    -37.812941     144.985883               15   
3             92.59    -37.811748     144.938192               25   
4             95.62    -37.818393     144.967034                5   
5             84.09    -37.801502     144.966104                0   
6             77.51    -37.823700     144.956314                0   
7             67.27    -37.817412     144.967590                5   
8            105.50    -37.807081     144.967288               15   
9             65.39    -37.810164     144.966819               15   

   order_total  season  is_expedited_delivery  distance_to_nearest_warehouse  \
0      6156.22  Autumn                   True                         0.4425   
1     12296.20  Autumn                  False                         0.6547   
2      3742.25     NaN                   True                         0.8852   
3      8297.59  Summer                   True                         0.7875   
4     20710.62  Spring                   True                         0.2225   
5      6904.09  Autumn                   True                         1.9268   
6      8567.51  Summer                  False                         1.2953   
7     30885.27  Summer                  False                         0.2169   
8      5970.50  Spring                   True                         1.2971   
9      6189.64  Autumn                  False                         0.9688   

                              latest_customer_review is_happy_customer  
0  to many adds in ui it has adds, sends info to ...             False  
1  s10e haven't had this phone but a short time b...              True  
2  best bang for your buck! great overall android...              True  
3  good it was a gift for my niece. it to her in ...              True  
4  excellent product excellent product, was recei...              True  
5  i love it very nice and good product would rec...              True  
6  great phone great phone i am still using that ...              True  
7  five stars it was a gift, my nephew is loving it.              True  
8  definitively would buy again. has been great t...              True  
9  my best snapchat phone my snapchats are crysta...              True

In [929]:
# using display for better formatting in Notebook , use print usually
display(df_missing_data.describe())

order_price  delivery_charges  customer_lat  customer_long  \
count    490.000000        500.000000    490.000000     490.000000   
mean   13217.867347         77.683380    -37.812597     144.966114   
std     7552.951453         14.566667      0.007593       0.021054   
min      580.000000         46.200000    -37.828216     144.921217   
25%     7140.000000         66.975000    -37.818755     144.951541   
50%    12180.000000         77.275000    -37.813053     144.963409   
75%    18327.500000         85.217500    -37.806714     144.980157   
max    37300.000000        110.990000    -37.788265     145.017015   

       coupon_discount   order_total  distance_to_nearest_warehouse  
count       500.000000    490.000000                     490.000000  
mean         11.190000  11939.841020                       1.077335  
std           8.734213   7032.605471                       0.509914  
min           0.000000    568.640000                       0.054900  
25%           5.000000   6486.542500                       0.723625  
50%          10.000000  10756.615000                       1.043350  
75%          15.000000  16315.555000                       1.389525  
max          25.000000  37362.470000                       3.138800

In [930]:
print(df_missing_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       500 non-null    object 
 1   customer_id                    500 non-null    object 
 2   date                           500 non-null    object 
 3   nearest_warehouse              490 non-null    object 
 4   shopping_cart                  500 non-null    object 
 5   order_price                    490 non-null    float64
 6   delivery_charges               500 non-null    float64
 7   customer_lat                   490 non-null    float64
 8   customer_long                  490 non-null    float64
 9   coupon_discount                500 non-null    int64  
 10  order_total                    490 non-null    float64
 11  season                         490 non-null    object 
 12  is_expedited_delivery          500 non-null    boo

In [931]:
print(df_missing_data.shape)

(500, 16)


In [932]:
print(df_missing_data.isna().any())

order_id                         False
customer_id                      False
date                             False
nearest_warehouse                 True
shopping_cart                    False
order_price                       True
delivery_charges                 False
customer_lat                      True
customer_long                     True
coupon_discount                  False
order_total                       True
season                            True
is_expedited_delivery            False
distance_to_nearest_warehouse     True
latest_customer_review           False
is_happy_customer                 True
dtype: bool


In [933]:
print(df_missing_data.isnull().sum())

order_id                          0
customer_id                       0
date                              0
nearest_warehouse                10
shopping_cart                     0
order_price                      10
delivery_charges                  0
customer_lat                     10
customer_long                    10
coupon_discount                   0
order_total                      10
season                           10
is_expedited_delivery             0
distance_to_nearest_warehouse    10
latest_customer_review            0
is_happy_customer                10
dtype: int64


### df_warehouses dataframe review

In [934]:
type(df_warehouses)

pandas.core.frame.DataFrame

In [935]:
print(df_warehouses.columns)

Index(['names', 'lat', 'lon'], dtype='object')


In [936]:
type(df_warehouses)

pandas.core.frame.DataFrame

In [937]:
df_warehouses.head()

names        lat         lon
0  Nickolson -37.818595  144.969551
1   Thompson -37.812673  144.947069
2     Bakers -37.809996  144.995232

In [938]:
# using display for better formatting in Notebook , use print usually
display(df_warehouses.describe())

lat         lon
count   3.000000    3.000000
mean  -37.813755  144.970617
std     0.004400    0.024099
min   -37.818595  144.947069
25%   -37.815634  144.958310
50%   -37.812673  144.969551
75%   -37.811335  144.982392
max   -37.809996  144.995232

In [939]:
print(df_warehouses.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   names   3 non-null      object 
 1   lat     3 non-null      float64
 2   lon     3 non-null      float64
dtypes: float64(2), object(1)
memory usage: 200.0+ bytes
None


In [940]:
print(df_warehouses.shape)

(3, 3)


In [941]:
print(df_warehouses.isna().any())

names    False
lat      False
lon      False
dtype: bool


In [942]:
print(df_warehouses.isnull().sum())

names    0
lat      0
lon      0
dtype: int64


## Re-import from CSV 
***

In [943]:
# read the CSV from the local Jupyter Notebook directory 
# parsing dates on import since dates were objects change datatype on order_price
df_dirty_data = pd.read_csv(dirty_data, parse_dates=['date'], dtype={'order_price': 'float'}) 
df_missing_data = pd.read_csv(missing_data, parse_dates=['date']) 

#  cant use dtype={'is_happy_customer': 'bool'} because is_happy_customer contains NaN 

In [944]:
print(df_dirty_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       500 non-null    object        
 1   customer_id                    500 non-null    object        
 2   date                           500 non-null    datetime64[ns]
 3   nearest_warehouse              500 non-null    object        
 4   shopping_cart                  500 non-null    object        
 5   order_price                    500 non-null    float64       
 6   delivery_charges               500 non-null    float64       
 7   customer_lat                   500 non-null    float64       
 8   customer_long                  500 non-null    float64       
 9   coupon_discount                500 non-null    int64         
 10  order_total                    500 non-null    float64       
 11  season             

In [945]:
print(df_missing_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       500 non-null    object        
 1   customer_id                    500 non-null    object        
 2   date                           500 non-null    datetime64[ns]
 3   nearest_warehouse              490 non-null    object        
 4   shopping_cart                  500 non-null    object        
 5   order_price                    490 non-null    float64       
 6   delivery_charges               500 non-null    float64       
 7   customer_lat                   490 non-null    float64       
 8   customer_long                  490 non-null    float64       
 9   coupon_discount                500 non-null    int64         
 10  order_total                    490 non-null    float64       
 11  season             

In [946]:
# check for orderid uniqueness in df_dirty_data
duplicate_dirty_data = df_dirty_data[df_dirty_data.duplicated(['order_id'])]
print(duplicate_dirty_data)

Empty DataFrame
Columns: [order_id, customer_id, date, nearest_warehouse, shopping_cart, order_price, delivery_charges, customer_lat, customer_long, coupon_discount, order_total, season, is_expedited_delivery, distance_to_nearest_warehouse, latest_customer_review, is_happy_customer]
Index: []


In [947]:
# check for orderid uniqueness in df_missing_data
duplicate_missing_data = df_missing_data[df_missing_data.duplicated(['order_id'])]
print(duplicate_missing_data)

Empty DataFrame
Columns: [order_id, customer_id, date, nearest_warehouse, shopping_cart, order_price, delivery_charges, customer_lat, customer_long, coupon_discount, order_total, season, is_expedited_delivery, distance_to_nearest_warehouse, latest_customer_review, is_happy_customer]
Index: []


In [948]:
# create an order_id index on df_dirty_data
df_dirty_data.set_index('order_id',inplace=True)

In [949]:
# create an order_id index on df_missing_data
df_missing_data.set_index('order_id',inplace=True)

In [950]:
df_dirty_data.index

Index(['ORD182494', 'ORD395518', 'ORD494479', 'ORD019224', 'ORD104032',
       'ORD146760', 'ORD337984', 'ORD072312', 'ORD377837', 'ORD462194',
       ...
       'ORD182549', 'ORD435271', 'ORD082002', 'ORD034351', 'ORD379700',
       'ORD475510', 'ORD086060', 'ORD079320', 'ORD026546', 'ORD085447'],
      dtype='object', name='order_id', length=500)

In [951]:
df_missing_data.index

Index(['ORD382112', 'ORD378488', 'ORD279446', 'ORD277196', 'ORD116193',
       'ORD005004', 'ORD296379', 'ORD447851', 'ORD078449', 'ORD018258',
       ...
       'ORD273851', 'ORD329425', 'ORD277938', 'ORD297371', 'ORD218866',
       'ORD289820', 'ORD425999', 'ORD252675', 'ORD215989', 'ORD414852'],
      dtype='object', name='order_id', length=500)

## Detect and fix errors in dirty_data
***

## Fix missing values in missing_data
***

In [907]:
# show missing values
print(df_missing_data.isnull().sum())

customer_id                       0
date                              0
nearest_warehouse                 0
shopping_cart                     0
order_price                      10
delivery_charges                  0
customer_lat                     10
customer_long                    10
coupon_discount                   0
order_total                      10
season                           10
is_expedited_delivery             0
distance_to_nearest_warehouse    10
latest_customer_review            0
is_happy_customer                10
dtype: int64


### missing 'nearest_warehouse'

In [952]:
# getting rows for the missing values for nearest_warehouse
nearest_warehouse_to_update = df_missing_data[df_missing_data['nearest_warehouse'].isnull()]

# using display command for better formatting in Notebook , use print usually
display(nearest_warehouse_to_update)

customer_id       date nearest_warehouse  \
order_id                                               
ORD447851  ID0591430562 2019-02-07               NaN   
ORD256447  ID1404216319 2019-12-13               NaN   
ORD159597  ID2632208068 2019-08-18               NaN   
ORD474381  ID2776391626 2019-12-11               NaN   
ORD372144  ID0814520160 2019-12-11               NaN   
ORD052805  ID4234925766 2019-05-19               NaN   
ORD222346  ID0368376953 2019-10-22               NaN   
ORD229269  ID0582156569 2019-12-31               NaN   
ORD433678  ID0176736472 2019-09-04               NaN   
ORD075737  ID2702095693 2019-10-22               NaN   

                                               shopping_cart  order_price  \
order_id                                                                    
ORD447851  [('Alcon 10', 2), ('Toshika 750', 2), ('Univer...      32440.0   
ORD256447  [('Candle Inferno', 1), ('Olivia x460', 1), ('...       6015.0   
ORD159597  [('Lucent 330S', 2), ('Candle Inferno', 2), ('...       9630.0   
ORD474381  [('Olivia x460', 2), ('Thunder line', 2), ('Ca...       7670.0   
ORD372144  [('iAssist Line', 2), ('Olivia x460', 1), ('Al...      18945.0   
ORD052805  [('Universe Note', 2), ('Candle Inferno', 2), ...       9940.0   
ORD222346  [('Olivia x460', 2), ('Universe Note', 1), ('L...       8360.0   
ORD229269  [('Alcon 10', 1), ('pearTV', 1), ('iAssist Lin...      21805.0   
ORD433678  [('Olivia x460', 1), ('iAssist Line', 1), ('pe...      16070.0   
ORD075737  [('iAssist Line', 2), ('pearTV', 2), ('Toshika...      25710.0   

           delivery_charges  customer_lat  customer_long  coupon_discount  \
order_id                                                                    
ORD447851             67.27    -37.817412     144.967590                5   
ORD256447             77.47    -37.801138     144.988605               25   
ORD159597             80.40    -37.819355     144.989577               10   
ORD474381             75.06    -37.810971     144.963102               25   
ORD372144             96.53    -37.812604     144.957172               15   
ORD052805             68.65    -37.804575     144.967468               15   
ORD222346             81.02    -37.802945     144.981699                0   
ORD229269             97.96    -37.788265     144.929202               25   
ORD433678             97.81    -37.814120     144.968115                0   
ORD075737             73.29    -37.810628     144.990476               15   

           order_total  season  is_expedited_delivery  \
order_id                                                
ORD447851     30885.27  Summer                  False   
ORD256447      4588.72  Summer                  False   
ORD159597      8747.40  Winter                   True   
ORD474381      5827.56  Summer                  False   
ORD372144     16199.78  Summer                   True   
ORD052805      8517.65  Autumn                   True   
ORD222346      8441.02  Spring                  False   
ORD229269     16451.71  Summer                  False   
ORD433678     16167.81  Spring                   True   
ORD075737     21926.79  Spring                  False   

           distance_to_nearest_warehouse  \
order_id                                   
ORD447851                         0.2169   
ORD256447                         1.1454   
ORD159597                         1.1544   
ORD474381                         1.0207   
ORD372144                         0.8885   
ORD052805                         1.5714   
ORD222346                         1.4257   
ORD229269                         3.1388   
ORD433678                         0.5139   
ORD075737                         0.4241   

                                      latest_customer_review is_happy_customer  
order_id                                                                        
ORD447851  five stars it was a gift, my nephew is loving it.              True  
ORD256447  i love this phone i l

In [953]:
# checking the missing values for nearest_warehouse
nearest_warehouse_indexto_update = df_missing_data[df_missing_data['nearest_warehouse'].isnull()].index

print(nearest_warehouse_indexto_update)

Index(['ORD447851', 'ORD256447', 'ORD159597', 'ORD474381', 'ORD372144',
       'ORD052805', 'ORD222346', 'ORD229269', 'ORD433678', 'ORD075737'],
      dtype='object', name='order_id')


In [958]:
# updating missing values for season
for i, row in nearest_warehouse_to_update.iterrows():
    df_missing_data.at[i,'nearest_warehouse'] = get_missing_nearest_warehouse(df_missing_data.at[i,'customer_lat'],df_missing_data.at[i,'customer_long'])[0]

In [959]:
# no more missing values for season
print(df_missing_data[df_missing_data['nearest_warehouse'].isnull()])

Empty DataFrame
Columns: [customer_id, date, nearest_warehouse, shopping_cart, order_price, delivery_charges, customer_lat, customer_long, coupon_discount, order_total, season, is_expedited_delivery, distance_to_nearest_warehouse, latest_customer_review, is_happy_customer]
Index: []


In [960]:
#check the indexes for the update 'nearest_warehouse' --  all done
df_missing_data.loc[nearest_warehouse_indexto_update]

customer_id       date nearest_warehouse  \
order_id                                               
ORD447851  ID0591430562 2019-02-07         Nickolson   
ORD256447  ID1404216319 2019-12-13            Bakers   
ORD159597  ID2632208068 2019-08-18            Bakers   
ORD474381  ID2776391626 2019-12-11         Nickolson   
ORD372144  ID0814520160 2019-12-11          Thompson   
ORD052805  ID4234925766 2019-05-19         Nickolson   
ORD222346  ID0368376953 2019-10-22            Bakers   
ORD229269  ID0582156569 2019-12-31          Thompson   
ORD433678  ID0176736472 2019-09-04         Nickolson   
ORD075737  ID2702095693 2019-10-22            Bakers   

                                               shopping_cart  order_price  \
order_id                                                                    
ORD447851  [('Alcon 10', 2), ('Toshika 750', 2), ('Univer...      32440.0   
ORD256447  [('Candle Inferno', 1), ('Olivia x460', 1), ('...       6015.0   
ORD159597  [('Lucent 330S', 2), ('Candle Inferno', 2), ('...       9630.0   
ORD474381  [('Olivia x460', 2), ('Thunder line', 2), ('Ca...       7670.0   
ORD372144  [('iAssist Line', 2), ('Olivia x460', 1), ('Al...      18945.0   
ORD052805  [('Universe Note', 2), ('Candle Inferno', 2), ...       9940.0   
ORD222346  [('Olivia x460', 2), ('Universe Note', 1), ('L...       8360.0   
ORD229269  [('Alcon 10', 1), ('pearTV', 1), ('iAssist Lin...      21805.0   
ORD433678  [('Olivia x460', 1), ('iAssist Line', 1), ('pe...      16070.0   
ORD075737  [('iAssist Line', 2), ('pearTV', 2), ('Toshika...      25710.0   

           delivery_charges  customer_lat  customer_long  coupon_discount  \
order_id                                                                    
ORD447851             67.27    -37.817412     144.967590                5   
ORD256447             77.47    -37.801138     144.988605               25   
ORD159597             80.40    -37.819355     144.989577               10   
ORD474381             75.06    -37.810971     144.963102               25   
ORD372144             96.53    -37.812604     144.957172               15   
ORD052805             68.65    -37.804575     144.967468               15   
ORD222346             81.02    -37.802945     144.981699                0   
ORD229269             97.96    -37.788265     144.929202               25   
ORD433678             97.81    -37.814120     144.968115                0   
ORD075737             73.29    -37.810628     144.990476               15   

           order_total  season  is_expedited_delivery  \
order_id                                                
ORD447851     30885.27  Summer                  False   
ORD256447      4588.72  Summer                  False   
ORD159597      8747.40  Winter                   True   
ORD474381      5827.56  Summer                  False   
ORD372144     16199.78  Summer                   True   
ORD052805      8517.65  Autumn                   True   
ORD222346      8441.02  Spring                  False   
ORD229269     16451.71  Summer                  False   
ORD433678     16167.81  Spring                   True   
ORD075737     21926.79  Spring                  False   

           distance_to_nearest_warehouse  \
order_id                                   
ORD447851                         0.2169   
ORD256447                         1.1454   
ORD159597                         1.1544   
ORD474381                         1.0207   
ORD372144                         0.8885   
ORD052805                         1.5714   
ORD222346                         1.4257   
ORD229269                         3.1388   
ORD433678                         0.5139   
ORD075737                         0.4241   

                                      latest_customer_review is_happy_customer  
order_id                                                                        
ORD447851  five stars it was a gift, my nephew is loving it.              True  
ORD256447  i love this phone i l

Empty DataFrame
Columns: [customer_id, date, nearest_warehouse, shopping_cart, order_price, delivery_charges, customer_lat, customer_long, coupon_discount, order_total, season, is_expedited_delivery, distance_to_nearest_warehouse, latest_customer_review, is_happy_customer]
Index: []

### missing 'order_price'

In [ ]:
# show missing 'order_price' rows
df_missing_data[df_missing_data['order_price'].isnull()]

In [ ]:
# getting the index for the missing values for order_price
order_price_indexto_update = df_missing_data[df_missing_data['order_price'].isnull()].index

In [ ]:
# updating the missing values for order_price
for i in order_price_indexto_update:
    df_missing_data.at[i,'order_price'] = get_missing_order_price(df_missing_data.at[i,'order_total'],df_missing_data.at[i,'delivery_charges'],df_missing_data.at[i,'coupon_discount'])

In [ ]:
# no more missing values for order_price
df_missing_data[df_missing_data['order_price'].isnull()]

In [ ]:
# check the indexes 'order_price' --  all done
for i in order_price_indexto_update:
    print(df_missing_data.at[i,'order_price'])

In [ ]:
df_missing_data.filter(items=["order_price"])

### missing 'order_total'

In [ ]:
# show missing 'order_total' rows
df_missing_data[df_missing_data['order_total'].isnull()]

In [ ]:
# getting the index for the missing values for order_total
order_total_indexto_update = df_missing_data[df_missing_data['order_total'].isnull()].index

In [ ]:
# updating the missing values for order_total
for i in order_total_indexto_update:
    df_missing_data.at[i,'order_total'] = get_missing_order_price(df_missing_data.at[i,'order_price'],df_missing_data.at[i,'delivery_charges'],df_missing_data.at[i,'coupon_discount'])

In [ ]:
# no more missing values for order_total
df_missing_data[df_missing_data['order_total'].isnull()]

In [ ]:
# check the indexes 'order_total' --  all done
for i in order_total_indexto_update:
    print(df_missing_data.at[i,'order_total'])

In [ ]:
df_missing_data.filter(items=["order_total"])

### missing 'season'

In [871]:
# show missing 'season' rows
df_missing_data[df_missing_data['season'].isnull()]

customer_id       date nearest_warehouse  \
order_id                                               
ORD279446  ID0370751503 2019-03-24            Bakers   
ORD083901  ID5457962366 2019-09-14          Thompson   
ORD250242  ID0287630803 2019-10-13          Thompson   
ORD078069  ID1458418088 2019-01-12            Bakers   
ORD016552  ID0283334264 2019-10-20          Thompson   
ORD429692  ID0575539547 2019-10-27         Nickolson   
ORD434639  ID1449297341 2019-06-03         Nickolson   
ORD032506  ID0628177290 2019-06-28          Thompson   
ORD027479  ID0493051199 2019-06-19            Bakers   
ORD437147  ID0052450505 2019-10-31          Thompson   

                                               shopping_cart  order_price  \
order_id                                                                    
ORD279446      [('Candle Inferno', 2), ('Universe Note', 1)]       4310.0   
ORD083901  [('Toshika 750', 1), ('pearTV', 1), ('iStream'...      10780.0   
ORD250242               [('Olivia x460', 2), ('iStream', 2)]       2750.0   
ORD078069  [('Lucent 330S', 1), ('Toshika 750', 2), ('pea...      16180.0   
ORD016552  [('iAssist Line', 1), ('Universe Note', 1), ('...      10035.0   
ORD429692  [('iAssist Line', 1), ('Alcon 10', 2), ('pearT...      32745.0   
ORD434639  [('Thunder line', 1), ('iStream', 2), ('Univer...       9380.0   
ORD032506  [('iStream', 2), ('Alcon 10', 2), ('Candle Inf...      18630.0   
ORD027479  [('Candle Inferno', 2), ('Alcon 10', 1), ('iSt...       9960.0   
ORD437147             [('iAssist Line', 2), ('Alcon 10', 2)]      22350.0   

           delivery_charges  customer_lat  customer_long  coupon_discount  \
order_id                                                                    
ORD279446             78.75    -37.812941     144.985883               15   
ORD083901             78.41    -37.822479     144.939978               15   
ORD250242            100.85    -37.806362     144.946628                5   
ORD078069            101.81    -37.819670     145.010019                0   
ORD016552             83.29    -37.801340     144.956120               25   
ORD429692             79.86    -37.823686     144.982575               15   
ORD434639             51.44    -37.815768     144.980178               25   
ORD032506             61.78    -37.811359     144.947461                5   
ORD027479             63.76    -37.807695     144.989743                5   
ORD437147             85.96    -37.795479     144.936073               15   

           order_total season  is_expedited_delivery  \
order_id                                               
ORD279446      3742.25    NaN                   True   
ORD083901      9241.41    NaN                  False   
ORD250242      2713.35    NaN                   True   
ORD078069     16281.81    NaN                   True   
ORD016552      7609.54    NaN                  False   
ORD429692     27913.11    NaN                  False   
ORD434639      7086.44    NaN                  False   
ORD032506     17760.28    NaN                  False   
ORD027479      9525.76    NaN                  False   
ORD437147     19083.46    NaN                  False   

           distance_to_nearest_warehouse  \
order_id                                   
ORD279446                         0.8852   
ORD083901                         1.2571   
ORD250242                         0.7036   
ORD078069                         1.6884   
ORD016552                         1.4917   
ORD429692                         1.2778   
ORD434639                         0.9860   
ORD032506                         0.1503   
ORD027479                         0.5465   
ORD437147                         2.1445   

                                      latest_customer_review is_happy_customer  
order_id                                                                        
ORD279446  best bang for your buck! great overall android...              True  
ORD083901  excelent product excellent relati

In [872]:
# getting the distinct values for season
print(df_missing_data['season'].sort_values().unique())

['Autumn' 'Spring' 'Summer' 'Winter' nan]


In [873]:
# get the seasons months from the grouping of the orders..obviously Southern hemisphere - a random lat long shows its Melbourne, Australia area
df_missing_data.groupby([pd.Grouper(key='date', axis=0, freq='M'),'season']).sum().filter(items=['date','season'])
# summer = months 12,1,2
# Autumn = months 3,4,5
# Winter = months 6,7,8
# Spring = months 9,10,11

Empty DataFrame
Columns: []
Index: [(2019-01-31 00:00:00, Summer), (2019-02-28 00:00:00, Summer), (2019-03-31 00:00:00, Autumn), (2019-04-30 00:00:00, Autumn), (2019-05-31 00:00:00, Autumn), (2019-06-30 00:00:00, Winter), (2019-07-31 00:00:00, Winter), (2019-08-31 00:00:00, Winter), (2019-09-30 00:00:00, Spring), (2019-10-31 00:00:00, Spring), (2019-11-30 00:00:00, Spring), (2019-12-31 00:00:00, Summer)]

In [874]:
# getting rows for the missing values for season
season_to_update = df_missing_data[df_missing_data['season'].isnull()]

In [875]:
# getting the index for the missing values for season
season_to_update_index = df_missing_data[df_missing_data['season'].isnull()].index

In [887]:
# updating missing values for season
for i, row in season_indexto_update.iterrows():
    df_missing_data.at[i,'season'] = get_missing_season(row['date'])

In [888]:
# no more missing values for season
df_missing_data[df_missing_data['season'].isnull()]

customer_id       date nearest_warehouse  \
order_id                                               
ORD279446  ID0370751503 2019-03-24            Bakers   
ORD083901  ID5457962366 2019-09-14          Thompson   
ORD250242  ID0287630803 2019-10-13          Thompson   
ORD078069  ID1458418088 2019-01-12            Bakers   
ORD016552  ID0283334264 2019-10-20          Thompson   
ORD429692  ID0575539547 2019-10-27         Nickolson   
ORD434639  ID1449297341 2019-06-03         Nickolson   
ORD032506  ID0628177290 2019-06-28          Thompson   
ORD027479  ID0493051199 2019-06-19            Bakers   
ORD437147  ID0052450505 2019-10-31          Thompson   

                                               shopping_cart  order_price  \
order_id                                                                    
ORD279446      [('Candle Inferno', 2), ('Universe Note', 1)]       4310.0   
ORD083901  [('Toshika 750', 1), ('pearTV', 1), ('iStream'...      10780.0   
ORD250242               [('Olivia x460', 2), ('iStream', 2)]       2750.0   
ORD078069  [('Lucent 330S', 1), ('Toshika 750', 2), ('pea...      16180.0   
ORD016552  [('iAssist Line', 1), ('Universe Note', 1), ('...      10035.0   
ORD429692  [('iAssist Line', 1), ('Alcon 10', 2), ('pearT...      32745.0   
ORD434639  [('Thunder line', 1), ('iStream', 2), ('Univer...       9380.0   
ORD032506  [('iStream', 2), ('Alcon 10', 2), ('Candle Inf...      18630.0   
ORD027479  [('Candle Inferno', 2), ('Alcon 10', 1), ('iSt...       9960.0   
ORD437147             [('iAssist Line', 2), ('Alcon 10', 2)]      22350.0   

           delivery_charges  customer_lat  customer_long  coupon_discount  \
order_id                                                                    
ORD279446             78.75    -37.812941     144.985883               15   
ORD083901             78.41    -37.822479     144.939978               15   
ORD250242            100.85    -37.806362     144.946628                5   
ORD078069            101.81    -37.819670     145.010019                0   
ORD016552             83.29    -37.801340     144.956120               25   
ORD429692             79.86    -37.823686     144.982575               15   
ORD434639             51.44    -37.815768     144.980178               25   
ORD032506             61.78    -37.811359     144.947461                5   
ORD027479             63.76    -37.807695     144.989743                5   
ORD437147             85.96    -37.795479     144.936073               15   

           order_total season  is_expedited_delivery  \
order_id                                               
ORD279446      3742.25    NaN                   True   
ORD083901      9241.41    NaN                  False   
ORD250242      2713.35    NaN                   True   
ORD078069     16281.81    NaN                   True   
ORD016552      7609.54    NaN                  False   
ORD429692     27913.11    NaN                  False   
ORD434639      7086.44    NaN                  False   
ORD032506     17760.28    NaN                  False   
ORD027479      9525.76    NaN                  False   
ORD437147     19083.46    NaN                  False   

           distance_to_nearest_warehouse  \
order_id                                   
ORD279446                         0.8852   
ORD083901                         1.2571   
ORD250242                         0.7036   
ORD078069                         1.6884   
ORD016552                         1.4917   
ORD429692                         1.2778   
ORD434639                         0.9860   
ORD032506                         0.1503   
ORD027479                         0.5465   
ORD437147                         2.1445   

                                      latest_customer_review is_happy_customer  
order_id                                                                        
ORD279446  best bang for your buck! great overall android...              True  
ORD083901  excelent product excellent relati

In [ ]:
#check the indexes for the update 'season' --  all done
for row in season_to_update_index.iterrows():
    row['season']

### missing 'distance_to_nearest_warehouse'

In [ ]:
# show missing 'distance_to_nearest_warehouse' rows
df_missing_data[df_missing_data['distance_to_nearest_warehouse'].isnull()]

In [ ]:
# getting the index for the missing values for nearest_warehouse
distance_to_nearest_warehouse_indexto_update = df_missing_data[df_missing_data['distance_to_nearest_warehouse'].isnull()].index

In [ ]:
# updating  missing values for nearest_warehouse
for i in distance_to_nearest_warehouse_indexto_update:
    df_missing_data.at[i,'distance_to_nearest_warehouse'] = get_missing_nearest_warehouse(df_missing_data.at[i,'customer_lat'],df_missing_data.at[i,'customer_long'])[1]

In [ ]:
# no more missing values for nearest_warehouse
df_missing_data[df_missing_data['distance_to_nearest_warehouse'].isnull()].index

In [ ]:
# check the indexes 'nearest_warehouse' --  all done
for i in nearest_warehouse_indexto_update:
    print(df_missing_data.at[i,'distance_to_nearest_warehouse'])

In [ ]:
df_missing_data.filter(items=["distance_to_nearest_warehouse"])

### missing 'customer_lat'

In [ ]:
# show missing 'customer_lat' rows
df_missing_data[df_missing_data['customer_lat'].isnull()]

In [ ]:
# getting the index for the missing values for customer_lat
customer_lat_indexto_update = df_missing_data[df_missing_data['customer_lat'].isnull()].index
customer_lat_indexto_update

In [ ]:
df_missing_data[df_missing_data.duplicated(['customer_id'])]

### missing 'customer_long'

In [ ]:
# show missing 'customer_long' rows
df_missing_data[df_missing_data['customer_long'].isnull()]

In [ ]:
# getting the index for the missing values for customer_long
customer_long_indexto_update = df_missing_data[df_missing_data['customer_long'].isnull()].index
customer_long_indexto_update

## Analyse
***

In [ ]:
print(df_missing_data.columns)

## Visualise
***